# Imports

In [1]:
# !pip install pandas pyarrow tensorflow h5py 

# !pip install numpy --upgrade --user

# !pip install tensorflow --upgrade --user

# !pip install keras --upgrade --user

# !pip install dtw-python

# !pip install scipy

In [2]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import scipy.fft 
import math


from pathlib import Path
from dtw import *

2023-08-19 17:33:39.870121: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 17:33:39.986560: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-19 17:33:39.989163: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-19 17:33:39.989175: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# Variables

In [3]:
root_data_eeg_path = Path(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/15_sec/')
root_data_ecg_path = Path(r'/media/s792288955/TOSHIBA EXT/Andrea/Kaggle/ECG-Dataset/')
root_data_emg_path = Path(r'/media/s792288955/TOSHIBA EXT/Andrea/Kaggle/ClassifyGestures_EMG/')

# Datasets

In [4]:
#EEG
#df_data_eeg_ori = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V1_01_tcp_ar.parquet')

# df_data_eeg_ori_v1_01_tcp_ar = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V1_01_tcp_ar.parquet')
# df_data_eeg_ori_v1_02_tcp_le = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V1_02_tcp_le.parquet')
# df_data_eeg_ori_v1_03_tcp_ar_a = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V1_03_tcp_ar_a.parquet')
# df_data_eeg_ori_v1_04_tcp_le_a = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V1_04_tcp_le_a.parquet')

# df_data_eeg_ori_v2_01_tcp_ar = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V2_01_tcp_ar.parquet')
# df_data_eeg_ori_v2_03_tcp_ar_a = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/10_sec/TUEG_V2_03_tcp_ar_a.parquet')


df_data_eeg_ori_v1_01_tcp_ar = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/15_sec/v1_1_0_01_tcp_ar.parquet')
df_data_eeg_ori_v1_02_tcp_le = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/15_sec/v1_1_0_02_tcp_le.parquet')
df_data_eeg_ori_v1_03_tcp_ar_a = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/15_sec/v1_1_0_03_tcp_ar_a.parquet')
df_data_eeg_ori_v1_04_tcp_le_a = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/15_sec/v1_1_0_04_tcp_le_a.parquet')

df_data_eeg_ori_v2_01_tcp_ar = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/15_sec/v1_2_0_01_tcp_ar.parquet')
df_data_eeg_ori_v2_03_tcp_ar_a = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/15_sec/v1_2_0_03_tcp_ar_a.parquet')

df_data_eeg_ori = pd.concat([df_data_eeg_ori_v1_01_tcp_ar
                           , 
                           df_data_eeg_ori_v1_02_tcp_le
                           ,
                           df_data_eeg_ori_v1_03_tcp_ar_a
                           , 
                           df_data_eeg_ori_v1_04_tcp_le_a
                           ,
                           df_data_eeg_ori_v2_01_tcp_ar
                           , 
                           df_data_eeg_ori_v2_03_tcp_ar_a
                            ])

# =-=-=-=-=-=-=-=-=---=-=-=-=--=-=-=-=

# 240 seconds
# df_data_eeg_ori_v1_01_tcp_ar_01 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_01.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_02 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_02.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_03 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_03.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_04 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_04.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_05 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_05.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_06 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_06.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_07 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_07.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_08 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_08.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_09 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_09.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_10 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_10.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_11 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_11.parquet')
# df_data_eeg_ori_v1_01_tcp_ar_12 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_01_tcp_ar_12.parquet')


# df_data_eeg_ori_v1_02_tcp_le_01 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_02_tcp_le_01.parquet')
# df_data_eeg_ori_v1_02_tcp_le_02 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_02_tcp_le_02.parquet')
# df_data_eeg_ori_v1_02_tcp_le_03 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_02_tcp_le_03.parquet')
# df_data_eeg_ori_v1_02_tcp_le_04 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_02_tcp_le_04.parquet')
# df_data_eeg_ori_v1_02_tcp_le_05 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_02_tcp_le_05.parquet')
# df_data_eeg_ori_v1_02_tcp_le_06 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_02_tcp_le_06.parquet')
# df_data_eeg_ori_v1_02_tcp_le_07 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_02_tcp_le_07.parquet')

# df_data_eeg_ori_v1_03_tcp_ar_a_01 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_03_tcp_ar_a_01.parquet')
# df_data_eeg_ori_v1_03_tcp_ar_a_02 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_03_tcp_ar_a_02.parquet')
# df_data_eeg_ori_v1_03_tcp_ar_a_03 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_03_tcp_ar_a_03.parquet')
# df_data_eeg_ori_v1_03_tcp_ar_a_04 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_03_tcp_ar_a_04.parquet')
# df_data_eeg_ori_v1_03_tcp_ar_a_05 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_03_tcp_ar_a_05.parquet')
# df_data_eeg_ori_v1_03_tcp_ar_a_06 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_03_tcp_ar_a_06.parquet')
# df_data_eeg_ori_v1_03_tcp_ar_a_07 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_03_tcp_ar_a_07.parquet')

# df_data_eeg_ori_v1_04_tcp_le_a_01 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V1_04_tcp_le_a_01.parquet')

# df_data_eeg_ori_v2_01_tcp_ar_01 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V2_01_tcp_ar_01.parquet')
# df_data_eeg_ori_v2_01_tcp_ar_02 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V2_01_tcp_ar_02.parquet')
# df_data_eeg_ori_v2_01_tcp_ar_03 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V2_01_tcp_ar_03.parquet')
# df_data_eeg_ori_v2_01_tcp_ar_04 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V2_01_tcp_ar_04.parquet')
# df_data_eeg_ori_v2_01_tcp_ar_05 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V2_01_tcp_ar_05.parquet')
# df_data_eeg_ori_v2_01_tcp_ar_06 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V2_01_tcp_ar_06.parquet')
# df_data_eeg_ori_v2_01_tcp_ar_07 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V2_01_tcp_ar_07.parquet')

# df_data_eeg_ori_v2_03_tcp_ar_a_01 = pd.read_parquet(r'/media/s792288955/TOSHIBA EXT/Andrea/Datasets/240_sec/TUEG_V2_03_tcp_ar_a_01.parquet')


# df_data_eeg_ori = pd.concat([df_data_eeg_ori_v1_01_tcp_ar_01, df_data_eeg_ori_v1_01_tcp_ar_02,
#                              df_data_eeg_ori_v1_01_tcp_ar_03. df_data_eeg_ori_v1_01_tcp_ar_04,
#                              df_data_eeg_ori_v1_01_tcp_ar_05, df_data_eeg_ori_v1_01_tcp_ar_06,
#                              df_data_eeg_ori_v1_01_tcp_ar_07, df_data_eeg_ori_v1_01_tcp_ar_08,
#                              df_data_eeg_ori_v1_01_tcp_ar_09, df_data_eeg_ori_v1_01_tcp_ar_10,
#                              df_data_eeg_ori_v1_01_tcp_ar_11, df_data_eeg_ori_v1_01_tcp_ar_12,
#                              df_data_eeg_ori_v1_02_tcp_le_01, df_data_eeg_ori_v1_02_tcp_le_02,
#                              df_data_eeg_ori_v1_02_tcp_le_03, df_data_eeg_ori_v1_02_tcp_le_04,
#                              df_data_eeg_ori_v1_02_tcp_le_05, df_data_eeg_ori_v1_02_tcp_le_06,
#                              df_data_eeg_ori_v1_02_tcp_le_07, df_data_eeg_ori_v1_03_tcp_ar_a_01,
#                              df_data_eeg_ori_v1_03_tcp_ar_a_02, df_data_eeg_ori_v1_03_tcp_ar_a_03,
#                              df_data_eeg_ori_v1_03_tcp_ar_a_04, df_data_eeg_ori_v1_03_tcp_ar_a_05,
#                              df_data_eeg_ori_v1_03_tcp_ar_a_06, df_data_eeg_ori_v1_03_tcp_ar_a_07,
#                              df_data_eeg_ori_v1_04_tcp_le_a_01, df_data_eeg_ori_v2_01_tcp_ar_01,
#                              df_data_eeg_ori_v2_01_tcp_ar_02, df_data_eeg_ori_v2_01_tcp_ar_03,
#                              df_data_eeg_ori_v2_01_tcp_ar_04, df_data_eeg_ori_v2_01_tcp_ar_05,
#                              df_data_eeg_ori_v2_01_tcp_ar_06, df_data_eeg_ori_v2_01_tcp_ar_07
#                             ])


print(df_data_eeg_ori.shape)


(412400, 61)


In [5]:
#ECG
df_data_ecg_train = pd.read_csv(r'/media/s792288955/TOSHIBA EXT/Andrea/Kaggle/ECG-Dataset/mitbih_train.csv',
                          sep=',', header=None)

df_data_ecg_abnormal = pd.read_csv(r'/media/s792288955/TOSHIBA EXT/Andrea/Kaggle/ECG-Dataset/ptbdb_abnormal.csv',
                          sep=',', header=None)

df_data_ecg_normal = pd.read_csv(r'/media/s792288955/TOSHIBA EXT/Andrea/Kaggle/ECG-Dataset/ptbdb_normal.csv',
                          sep=',', header=None)


df_data_ecg_test_ori = pd.read_csv(r'/media/s792288955/TOSHIBA EXT/Andrea/Kaggle/ECG-Dataset/mitbih_test.csv',
                          sep=',', header=None)


df_data_ecg_ori = pd.concat([df_data_ecg_train, df_data_ecg_abnormal, df_data_ecg_normal, df_data_ecg_test_ori], axis=0)

In [6]:
#EMG
all_filenames = [i for i in root_data_emg_path.glob('*.{}'.format('csv'))]

df_data_emg_0_ori = pd.read_csv(all_filenames[0], 
                                sep=",", header=None)

df_data_emg_1_ori = pd.read_csv(all_filenames[1], 
                                sep=",", header=None)

df_data_emg_2_ori = pd.read_csv(all_filenames[2], 
                                 sep=",", header=None)

df_data_emg_3_ori = pd.read_csv(all_filenames[3], 
                                   sep=",", header=None)

df_data_emg_ori   = pd.concat([df_data_emg_0_ori,df_data_emg_1_ori,df_data_emg_2_ori,df_data_emg_3_ori], axis=0)


In [7]:
df_data_eeg_ori.shape

(412400, 61)

In [8]:
df_data_ecg_ori.shape

(123998, 188)

In [9]:
df_data_emg_ori.shape

(11678, 65)

# Statistics

In [10]:
df_data_eeg_ori = df_data_eeg_ori[["F3", "F4", "F7", "F8", "C3", "C4", "C3P", "C4P", "T1", "T2", "T3", 
                                   "T4", "P3", "P4", "O1", "O2", 'id_diagnose']].copy()


# y_train_eeg = df_data_eeg_ori['id_diagnose'].copy()

df_data_eeg_ori['id_diagnose'] = df_data_eeg_ori['id_diagnose'].fillna(10)

In [11]:
# print(df_data_eeg_ori['id_diagnose'].describe(include="all"))
# print(df_data_eeg_ori['id_diagnose'].agg(['sum', 'min', 'max', np.mean, np.median]))
# print(f"Tem  : {pd.isna(df_data_eeg_ori['id_diagnose'])}")

# df_data_eeg_ori_v1_01_tcp_ar_01
# print(df_data_eeg_ori_v1_01_tcp_ar_01['id_diagnose'].describe(include="all"))
# print(df_data_eeg_ori_v1_01_tcp_ar_01['id_diagnose'].agg(['sum', 'min', 'max', np.mean, np.median]))
# print(f"Tem  : {pd.isna(df_data_eeg_ori_v1_01_tcp_ar_01['id_diagnose']).sum()}")


In [12]:
print(df_data_eeg_ori.shape)
print("=-=-=-=-=-=-=-")
print(df_data_eeg_ori.columns.values)
print("=-=-=-=-=-=-=-")
print(df_data_eeg_ori.describe(include="all"))
print("=-=-=-=-=-=-=-")
print(df_data_eeg_ori['id_diagnose'].agg(['sum', 'min', 'max', np.mean, np.median]))
print("=-=-=-=-=-=-=-")
print(df_data_eeg_ori['id_diagnose'].head(725))
print(df_data_eeg_ori.head(31790))
print("=-=-=-=-=-=-=-")
print(df_data_eeg_ori.id_diagnose.unique())

(412400, 17)
=-=-=-=-=-=-=-
['F3' 'F4' 'F7' 'F8' 'C3' 'C4' 'C3P' 'C4P' 'T1' 'T2' 'T3' 'T4' 'P3' 'P4'
 'O1' 'O2' 'id_diagnose']
=-=-=-=-=-=-=-
                  F3             F4             F7             F8  \
count  412400.000000  412400.000000  412400.000000  412400.000000   
mean      -22.608099     -19.181707     -15.362952     -11.560353   
std       584.182985     569.357885     536.909606     533.747543   
min     -5482.447311   -5482.447311   -5482.447311   -5482.447311   
25%       -27.465835     -25.761915     -33.123605     -28.381362   
50%       -10.681158     -10.536602     -13.046269      -9.887586   
75%        13.427741      13.427741      13.506709      16.784677   
max      5482.288000    5482.288000    5482.288000    5482.288000   

                  C3             C4            C3P            C4P  \
count  412400.000000  412400.000000  411824.000000  411824.000000   
mean      -26.074262     -24.646242    -190.030288    -207.322211   
std       537.005300     522.

In [13]:
print("id_patient =" , pd.isna(df_data_eeg_ori.id_patient).sum())
print("id_patient_version = " , pd.isna(df_data_eeg_ori.id_patient_version).sum())
print("token = " , pd.isna(df_data_eeg_ori.token).sum())
print("time = " , pd.isna(df_data_eeg_ori.time).sum())
print("id_diagnose = " , pd.isna(df_data_eeg_ori.id_diagnose).sum())
print("A1 = " , pd.isna(df_data_eeg_ori.A1).sum())
print("A2 = " , pd.isna(df_data_eeg_ori.A2).sum())
print("F3 = " , pd.isna(df_data_eeg_ori.F3).sum())
print("F4 = " , pd.isna(df_data_eeg_ori.F4).sum())
print("F7 = " , pd.isna(df_data_eeg_ori.F7).sum())
print("F8 = " , pd.isna(df_data_eeg_ori.F8).sum())
print("FP1 = " , pd.isna(df_data_eeg_ori.FP1).sum())
print("FP2 = " , pd.isna(df_data_eeg_ori.FP2).sum())
print("FZ = " , pd.isna(df_data_eeg_ori.FZ).sum())
print("T1 = " , pd.isna(df_data_eeg_ori.T1).sum())
print("T2 = " , pd.isna(df_data_eeg_ori.T2).sum())
print("T3 = " , pd.isna(df_data_eeg_ori.T3).sum())
print("T4 = " , pd.isna(df_data_eeg_ori.T4).sum())
print("T5 = " , pd.isna(df_data_eeg_ori.T5).sum())
print("T6 = " , pd.isna(df_data_eeg_ori.T6).sum())
print("C3 = " , pd.isna(df_data_eeg_ori.C3).sum())
print("C4 = " , pd.isna(df_data_eeg_ori.C4).sum())
print("C3P = " , pd.isna(df_data_eeg_ori.C3P).sum())
print("C4P = " , pd.isna(df_data_eeg_ori.C4P).sum())
print("CZ = " , pd.isna(df_data_eeg_ori.CZ).sum())
print("20 = " , pd.isna(df_data_eeg_ori['20']).sum())
print("21 = " , pd.isna(df_data_eeg_ori['21']).sum())
print("22 = " , pd.isna(df_data_eeg_ori['22']).sum())
print("23 = " , pd.isna(df_data_eeg_ori['23']).sum())
print("24 = " , pd.isna(df_data_eeg_ori['24']).sum())
print("25 = " , pd.isna(df_data_eeg_ori['25']).sum())
print("26 = " , pd.isna(df_data_eeg_ori['26']).sum())
print("27 = " , pd.isna(df_data_eeg_ori['27']).sum())
print("28 = " , pd.isna(df_data_eeg_ori['28']).sum())
print("29 = " , pd.isna(df_data_eeg_ori['29']).sum())
print("30 = " , pd.isna(df_data_eeg_ori['30']).sum())
print("31 = " , pd.isna(df_data_eeg_ori['31']).sum())
print("32 = " , pd.isna(df_data_eeg_ori['32']).sum())
print("DC1 = " , pd.isna(df_data_eeg_ori.DC1).sum())
print("DC2 = " , pd.isna(df_data_eeg_ori.DC2).sum())
print("DC3 = " , pd.isna(df_data_eeg_ori.DC3).sum())
print("DC4 = " , pd.isna(df_data_eeg_ori.DC4).sum())
print("DC5 = " , pd.isna(df_data_eeg_ori.DC5).sum())
print("DC6 = " , pd.isna(df_data_eeg_ori.DC6).sum())
print("DC7 = " , pd.isna(df_data_eeg_ori.DC7).sum())
print("DC8 = " , pd.isna(df_data_eeg_ori.DC8).sum())
print("EKG = " , pd.isna(df_data_eeg_ori.EKG).sum())
print("EKG1 = " , pd.isna(df_data_eeg_ori.EKG1).sum())
print("EMG = " , pd.isna(df_data_eeg_ori.EMG).sum())
print("O1 = " , pd.isna(df_data_eeg_ori.O1).sum())
print("O2 = " , pd.isna(df_data_eeg_ori.O2).sum())
print("OZ = " , pd.isna(df_data_eeg_ori.OZ).sum())
print("P3 = " , pd.isna(df_data_eeg_ori.P3).sum())
print("P4 = " , pd.isna(df_data_eeg_ori.P4).sum())
print("PG1 = " , pd.isna(df_data_eeg_ori.PG1).sum())
print("PG2 = " , pd.isna(df_data_eeg_ori.PG2).sum())
print("PZ = " , pd.isna(df_data_eeg_ori.PZ).sum())
print("ROC = " , pd.isna(df_data_eeg_ori.ROC).sum())
print("SP1 = " , pd.isna(df_data_eeg_ori.SP1).sum())
print("SP2 = " , pd.isna(df_data_eeg_ori.SP2).sum())
print("LOC = " , pd.isna(df_data_eeg_ori.LOC).sum())

AttributeError: 'DataFrame' object has no attribute 'id_patient'

In [ ]:
vet_no_diagnose_temp = []
vet_no_diagnose = []

for i, row in df_data_eeg_ori.iterrows():
    if pd.isna(row['id_diagnose']) == True:
        vet_no_diagnose_temp.append(row['id_patient'])
        
vet_no_diagnose = set(vet_no_diagnose_temp)   
vet_no_diagnose = list(vet_no_diagnose)   
print(vet_no_diagnose)

In [ ]:
for i, row in df_data_eeg_ori.iterrows():
    if row['id_patient'] == 21:
        print(row['id_patient_version'] , " -", row['id_patient'] , " -", row['id_diagnose'])

In [ ]:
# df_data_eeg_ori['PG2'].isnull

# print(df_data_eeg_ori['id_diagnose'].agg(['sum', 'min', 'max', np.mean, np.median]))

# print(np.isnan(df_data_eeg_ori['id_diagnose']))

print(df_data_eeg_ori['id_diagnose'].describe(include="all"))

#, "PG2"

In [14]:
print(df_data_ecg_ori.shape)
print("=-=-=-=-=-=-=-")
print(df_data_ecg_ori.columns.values)

(123998, 188)
=-=-=-=-=-=-=-
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187]


In [ ]:
print(df_data_emg_ori.shape)
print("=-=-=-=-=-=-=-")
print(df_data_emg_ori.columns.values)

### EEG

In [15]:
df_data_eeg_ori.agg(['sum', 'min', 'max', np.mean, np.median])

,F3,F4,F7,F8,C3,C4,C3P,C4P,T1,T2,T3,T4,P3,P4,O1,O2,id_diagnose
sum,-9.323580e+06,-7.910536e+06,-6.335681e+06,-4.767489e+06,-1.075303e+07,-1.016411e+07,-7.825903e+07,-8.538026e+07,-3.482417e+07,-3.116207e+07,-7.168155e+06,-8.132308e+06,-1.045174e+07,-1.004950e+07,-8.986829e+06,-6.139667e+06,868048.000000
min,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,-5.482447e+03,1.000000
max,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,5.482288e+03,10.000000
mean,-2.260810e+01,-1.918171e+01,-1.536295e+01,-1.156035e+01,-2.607426e+01,-2.464624e+01,-1.900303e+02,-2.073222e+02,-8.444270e+01,-7.556274e+01,-1.738156e+01,-1.971947e+01,-2.534371e+01,-2.436832e+01,-2.179154e+01,-1.488765e+01,2.104869
median,-1.068116e+01,-1.053660e+01,-1.304627e+01,-9.887586e+00,-1.020198e+01,-1.271165e+01,-6.758970e+01,-1.594435e+02,-2.141183e+01,-1.953126e+01,-1.647950e+01,-1.555594e+01,-1.338089e+01,-1.438476e+01,-1.120585e+01,-8.088025e+00,2.000000


### ECG

In [16]:
df_data_ecg_ori.agg(['sum', 'min', 'max', np.mean, np.median])

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
sum,111746.993648,93561.170944,52326.065295,27569.187950,25040.004545,26166.052482,25729.850563,25332.853537,25031.180116,24860.805722,...,557.664843,516.400538,476.768930,439.091177,411.568924,386.560155,353.933212,326.180399,310.280030,62322.000000
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000
mean,0.901200,0.754538,0.421991,0.222336,0.201939,0.211020,0.207502,0.204301,0.201868,0.200494,...,0.004497,0.004165,0.003845,0.003541,0.003319,0.003117,0.002854,0.002631,0.002502,0.502605
median,0.992308,0.819217,0.423664,0.166667,0.146534,0.158763,0.147482,0.148515,0.153846,0.152443,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### EMG

In [ ]:
df_data_emg_ori.agg(['sum', 'min', 'max', np.mean, np.median])